# xgboost

#### Load the packages and import the data

In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv("./Data Files/Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [2]:
data.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

#### Split data into an X DataFrame and y vector

In [3]:
X = data.drop("target", axis = 1)
y = data["target"]

#### Split the data into a train_set and test_set

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1111)

#### Center and Scale X

In [5]:
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
for i in X_train.columns:
    if X_train[i].dtypes in ["float64", "int64"]:
        m = X_train[i].mean()
        s = X_train[i].std()
        X_train_scaled[i] = (X_train[i] - m) / s
        X_test_scaled[i] = (X_test[i] - m) / s

#### Fit the base SVM Model

In [6]:
from sklearn.svm import SVC

svm_model = SVC(random_state = 1111)
svm_model.fit(X_train_scaled, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=1111, shrinking=True,
  tol=0.001, verbose=False)

#### Predict base SVM model on Test Set

In [7]:
y_pred = svm_model.predict(X_test_scaled)
pred_summary = X_test.copy()
pred_summary[y.name] = y_test  # dependent variable
pred_summary["y_pred"] = y_pred
pred_summary.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target,y_pred
472,14.920,14.93,96.45,686.9,0.08098,0.08549,0.05539,0.03221,0.1687,0.05669,...,112.00,906.6,0.1065,0.27910,0.3151,0.1147,0.2688,0.08273,1,1
175,8.671,14.45,54.42,227.2,0.09138,0.04276,0.00000,0.00000,0.1722,0.06724,...,58.36,259.2,0.1162,0.07057,0.0000,0.0000,0.2592,0.07848,1,1
538,7.729,25.49,47.98,178.8,0.08098,0.04878,0.00000,0.00000,0.1870,0.07285,...,57.17,248.0,0.1256,0.08340,0.0000,0.0000,0.3058,0.09938,1,1
550,10.860,21.48,68.51,360.5,0.07431,0.04227,0.00000,0.00000,0.1661,0.05948,...,74.08,412.3,0.1001,0.07348,0.0000,0.0000,0.2458,0.06592,1,1
44,13.170,21.81,85.42,531.5,0.09714,0.10470,0.08259,0.05252,0.1746,0.06177,...,105.50,740.7,0.1503,0.39040,0.3728,0.1607,0.3693,0.09618,0,0


#### Evaluate the base SVM Model

In [8]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print("Base SVM Model", "\n")
print(pd.DataFrame(confusion_matrix(y_test, y_pred)), 
      "      Accuracy:", round(accuracy_score(y_test, y_pred), 3), 
      "\n")
print(classification_report(y_test, y_pred))

Base SVM Model 

    0    1
0  64    3
1   1  103       Accuracy: 0.977 

             precision    recall  f1-score   support

          0       0.98      0.96      0.97        67
          1       0.97      0.99      0.98       104

avg / total       0.98      0.98      0.98       171



#### Use Cross Validation to tune the C, gamma, and kernel parameters

In [9]:
from sklearn.model_selection import GridSearchCV
parameters = {"C": [0.1, 1, 10, 100, 1000],  # Adjust these values over and over
              "gamma": [1, 0.1, 0.01, 0.001, 0.0001],
              "kernel": ['linear', 'poly', 'rbf', 'sigmoid']}
grid = GridSearchCV(SVC(), parameters, verbose = 2)
grid.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] .................... C=0.1, gamma=1, kernel=linear, total=   0.0s
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] .................... C=0.1, gamma=1, kernel=linear, total=   0.0s
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] .................... C=0.1, gamma=1, kernel=linear, total=   0.0s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ...................... C=0.1, gamma=1, kernel=poly, total=   0.0s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ...................... C=0.1, gamma=1, kernel=poly, total=   0.0s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ...................... C=0.1, gamma=1, kernel=poly, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .........

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ................. C=0.1, gamma=0.1, kernel=sigmoid, total=   0.0s
[CV] C=0.1, gamma=0.1, kernel=sigmoid ................................
[CV] ................. C=0.1, gamma=0.1, kernel=sigmoid, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] ................. C=0.1, gamma=0.01, kernel=linear, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] ................. C=0.1, gamma=0.01, kernel=linear, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] ................. C=0.1, gamma=0.01, kernel=linear, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=poly ..................................
[CV] ................... C=0.1, gamma=0.01, kernel=poly, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=poly ..................................
[CV] ................... C=0.1, gamma=0.01, kernel=poly, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=poly ..................................
[CV] .

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    2.5s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=2)

In [10]:
grid.best_params_

{'C': 0.1, 'gamma': 1, 'kernel': 'linear'}

#### Predict on test set using new parameters

In [11]:
grid_pred = grid.predict(X_test_scaled)
pred_summary = X_test.copy()
pred_summary[y.name] = y_test  # dependent variable
pred_summary["y_pred"] = grid_pred
pred_summary.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target,y_pred
472,14.920,14.93,96.45,686.9,0.08098,0.08549,0.05539,0.03221,0.1687,0.05669,...,112.00,906.6,0.1065,0.27910,0.3151,0.1147,0.2688,0.08273,1,1
175,8.671,14.45,54.42,227.2,0.09138,0.04276,0.00000,0.00000,0.1722,0.06724,...,58.36,259.2,0.1162,0.07057,0.0000,0.0000,0.2592,0.07848,1,1
538,7.729,25.49,47.98,178.8,0.08098,0.04878,0.00000,0.00000,0.1870,0.07285,...,57.17,248.0,0.1256,0.08340,0.0000,0.0000,0.3058,0.09938,1,1
550,10.860,21.48,68.51,360.5,0.07431,0.04227,0.00000,0.00000,0.1661,0.05948,...,74.08,412.3,0.1001,0.07348,0.0000,0.0000,0.2458,0.06592,1,1
44,13.170,21.81,85.42,531.5,0.09714,0.10470,0.08259,0.05252,0.1746,0.06177,...,105.50,740.7,0.1503,0.39040,0.3728,0.1607,0.3693,0.09618,0,0


#### Evaluate the model using new parameters

In [12]:
print("Refined SVM Model", "\n")
print(pd.DataFrame(confusion_matrix(y_test, y_pred)), 
      "      Accuracy:", round(accuracy_score(y_test, y_pred), 3), 
      "\n")
print(classification_report(y_test, y_pred))

Refined SVM Model 

    0    1
0  64    3
1   1  103       Accuracy: 0.977 

             precision    recall  f1-score   support

          0       0.98      0.96      0.97        67
          1       0.97      0.99      0.98       104

avg / total       0.98      0.98      0.98       171

